<a href="https://colab.research.google.com/github/FabioERodrigues/Sentiment-analysis-model-research/blob/main/FinalYear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikeras

Fabio Rodrigues
31016067

In [ ]:
import os
import re
import shutil
import tarfile
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
import spacy
import tensorflow as tf
from bs4 import BeautifulSoup
from collections import Counter
from scipy.stats import uniform
from sklearn.ensemble import (
    RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier,
    AdaBoostClassifier
)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    ConfusionMatrixDisplay
)
from sklearn.model_selection import (
    train_test_split, GridSearchCV, RandomizedSearchCV
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from transformers import (
    BertTokenizer, TFBertForSequenceClassification
)
from wordcloud import WordCloud, STOPWORDS
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split
import time

In [ ]:
movies = "/content/rotten_tomatoes_movies.csv"
movie_review = "/content/rotten_tomatoes_movie_reviews.csv"

In [ ]:
df1 = pd.read_csv(movies, encoding='latin-1')
df2 = pd.read_csv(movie_review, encoding='latin-1')

MERGING THE datasets

In [ ]:
# Performs an inner merge on the 'id' column
merged_df = pd.merge(df1, df2, on='id', how='inner')

# Displays the first few rows of the merged DataFrame
print(merged_df.head())

#Exploratory Data Analysis(EDA)



In [ ]:
merged_df.head()

In [ ]:
merged_df.info()

In [ ]:
merged_df.nunique()

In [ ]:
merged_df.isnull().sum()

In [ ]:
(merged_df.isnull().sum()/(len(merged_df)))*100 # perectange of null values in ecery column

In [ ]:
# REMOVING irrelavant COLUMNS

columns_to_drop = [
     'tomatoMeter', 'ratingContents',
    'runtimeMinutes', 'originalLanguage', 'director', 'writer',
    'boxOffice', 'distributor', 'soundMix', 'reviewState',
    'publicatioName', 'reviewUrl',  'releaseDateTheaters', 'releaseDateStreaming', 'reviewId', 'creationDate', 'criticName', 'rating', 'originalScore'
]

merged_df = merged_df.drop(columns=columns_to_drop)


In [ ]:
merged_df.head()

In [ ]:
merged_df['reviewText'] = merged_df['reviewText'].astype(str)

##Wordcloud

In [ ]:
# Concatenate reviews by sentiment
negative_reviews = ' '.join(merged_df[merged_df['scoreSentiment'] == "NEGATIVE"]['reviewText'])
positive_reviews = ' '.join(merged_df[merged_df['scoreSentiment'] == "POSITIVE"]['reviewText'])

# Generate word clouds
wordcloud_neg = WordCloud(width=800, height=400).generate(negative_reviews)
wordcloud_pos = WordCloud(width=800, height=400).generate(positive_reviews)

# Create subplots: 1 row, 2 columns
fig, axes = plt.subplots(1, 2, figsize=(15, 7))

# Plot Negative Reviews WordCloud
axes[0].imshow(wordcloud_neg, interpolation='bilinear')
axes[0].set_title('Class: Negative')
axes[0].axis('off')

# Plot Positive Reviews WordCloud
axes[1].imshow(wordcloud_pos, interpolation='bilinear')
axes[1].set_title('Class: Positive')
axes[1].axis('off')

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# words like good appear in negative class after looking into it i found a few example i found why this might be for example
#"It doesn't matter if a movie costs 300 million or only 300 dollars; good is good and bad is bad, and Bloodmask: The Possession of Nicole Lameroux is just plain bad."
#reviews like this might explain why good appears in  negative class
 #another example
#"In the comic context of A Gorgeous Bird Like Me, a good deal of the savour is lost simply because this femme comes over as much less fatale than may have been intended."
#Director Joel Schumacher, credited with toning down Ebbe Roe Smith's script, paces the film rivetingly, which goes to show that good film-making doesn't guarantee a good film. Falling Down is as sleek and efficient as a torture chamber.


##Distribution of Review length

In [ ]:
# Calculate review lengths
merged_df['review_length'] = merged_df['reviewText'].apply(lambda x: len(x.split()))

# Plotting the distribution of review lengths
plt.figure(figsize=(8, 6))
plt.hist(merged_df['review_length'], bins=50, color='c', edgecolor='k', alpha=0.7)
plt.title('Distribution of Review Lengths')
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.show()

## Sentiment distribution

In [ ]:
#Sentiment distribution
sentiment_counts = merged_df['scoreSentiment'].value_counts()

In [ ]:
#Sentiment distribution visulaised using pie chart
# Pie chart for sentiment distribution
plt.figure(figsize=(6, 6))
plt.pie(
    sentiment_counts,
    labels=sentiment_counts.index,
    autopct='%1.1f%%',
    colors=['lightgreen', 'lightcoral'],
    startangle=140
)
plt.title('Sentiment Distribution')
plt.axis('equal')  # Equal aspect ratio ensures the pie is drawn as a circle.
plt.show()

#Pre-Processing


In [ ]:
#Removing 0 t0 5 word reviews
print(merged_df.shape)
# Remove reviews with 5 word or less
merged_df = merged_df[merged_df['review_length'] > 5]

# Reset index
merged_df = merged_df.reset_index(drop=True)

# Verify the changes
print(merged_df.shape)

In [ ]:
merged_df = merged_df.dropna(subset=['reviewText'])
merged_df = merged_df.dropna(subset=['title'])
merged_df = merged_df.dropna(subset=['audienceScore'])
merged_df = merged_df.dropna(subset=['genre'])
merged_df = merged_df.dropna(subset=['scoreSentiment'])

# Double-check if any NaN values remain
print(merged_df.isnull().sum())
# Check if y has NaN values
print((merged_df.applymap(lambda x: x.strip() == "" if isinstance(x, str) else False)).sum())
print((merged_df.applymap(lambda x: x.strip() == "" if isinstance(x, str) else False)).sum())
print((merged_df.isin([None])).sum())
print((merged_df.isin(["NULL", "NA", "-"])).sum())

print(merged_df.dtypes)


In [ ]:
print(merged_df["scoreSentiment"].unique())

In [ ]:
merged_df.shape

In [ ]:
#Removing special characters for the reviewtext
#Preprocess the 'text' column
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)  # Remove hashtags symbol
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text

merged_df['reviewText'] = merged_df['reviewText'].apply(clean_text)

In [ ]:
#Sample 100,000 to tokenize for model testing

sampled_df = merged_df.sample(n=10000, random_state=42)

In [ ]:
sampled_df["scoreSentiment"] = sampled_df["scoreSentiment"].map({"POSITIVE": 1, "NEGATIVE": 0})

In [ ]:
sampled_df.dtypes

In [ ]:
print(sampled_df["scoreSentiment"].unique())

In [ ]:
print(sampled_df.isnull().sum())

In [ ]:
sampled_df.head(5)

#Machine learning methods

In [ ]:
#Traditional models

In [ ]:
nlp= spacy.load('en_core_web_sm')

##Negation, Lemmatization and tokenisation

In [ ]:
def process_text(s):
    doc = nlp(s)
    out = []
    negation = False
    for token in doc:
        if token.dep_ == 'neg':  # Detect negation
            negation = True
        elif token.is_punct:
            negation = False

        if not token.is_stop and not token.is_punct:
            lemma = token.lemma_.lower()
            if negation:
                lemma = f"not_{lemma}"
                negation = False  # Reset after handling
            out.append(lemma)
    return ' '.join(out)

In [ ]:
sampled_df['fltrTxt']= sampled_df['reviewText'].apply(process_text)

In [ ]:
# after first doing the cod there was overfitting which i found out due to getting 100% accuracy on some of ml models so some changes were made to combat this problem

In [ ]:
# Split the dataset into training and testing sets
X = sampled_df['fltrTxt']
y = sampled_df['scoreSentiment']

X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Now apply TF-IDF vectorization only on the training set and transform the test set
vct = TfidfVectorizer(max_features=10_000, stop_words='english', ngram_range=(1,2))
X_trn = vct.fit_transform(X_trn)
X_tst = vct.transform(X_tst)  # Transform test set with the same vectorizer


In [ ]:
def model_report(model, verbose=True):
    model.fit(X_trn, y_trn)

    # Start timing for training
    train_start = time.time()
    model.fit(X_trn, y_trn)
    train_end = time.time()
    train_time = train_end - train_start
    # Start timing for inference
    inference_start = time.time()
    y_pred = model.predict(X_tst)
    inference_end = time.time()
    inference_time = inference_end - inference_start
    trnScore = model.score(X_trn, y_trn)
    tstScore = model.score(X_tst, y_tst)
    cm = confusion_matrix(y_tst, y_pred)
    cr = classification_report(y_tst, y_pred)

    if verbose:
        print('Train Score: %f' % trnScore)
        print('Test Score: %f' % tstScore)
        print('Training Time: %.2f seconds' % train_time)
        print('Inference Time: %.2f seconds' % inference_time)
        print('Classification Report:\n', cr)
        ConfusionMatrixDisplay(cm).plot()
        plt.show()
        print()

    return {
        'trn': trnScore,
        'tst': tstScore,
        'cm': cm,
        'cr': cr,
        'train_time': train_time,
        'inference_time': inference_time
    }

In [ ]:
models_dict= {
	'LogisticRegression':     LogisticRegression(max_iter=10_000, class_weight='balanced'),
	'Support Vector':         SVC(class_weight='balanced'),
	'KNeighborsCLassifier':   KNeighborsClassifier(n_neighbors=15), # to fix overfiiting
	'DecisionTreeClassifier': DecisionTreeClassifier(class_weight='balanced', max_depth=10), # Prevent overfitting
	'RandomForestClassifier': RandomForestClassifier(class_weight='balanced'),
	'BaggingClassifier':      BaggingClassifier(),
	'ExtraTreesClassifier':   ExtraTreesClassifier(class_weight='balanced'),
	'AdaBoostClassifier':     AdaBoostClassifier(),
	'XGBClassifier':          XGBClassifier( n_estimators=100, max_depth=3),# Shallow trees help avoid overfitting
	'LGBMClassifier':         LGBMClassifier(class_weight='balanced'),
}
models= [{'name':k, 'obj':v} for k,v in models_dict.items()]

i= 0
for model in models:
	print("Evaluating %s..."%model['name'])
	print("%d/%d models"%(i, len(models)), end='\r')
	model.update(model_report(model['obj'], verbose=False))
	i+= 1
print("%d/%d models evaluated"%(i, len(models)))
print("done")

## Train score and test score for traditional machine learning models

In [ ]:
ml_results_df = pd.DataFrame({
	'Algorithm':           [model['name'] for model in models],
	'Train Score':         [model['trn']  for model in models],
	'Test Score':          [model['tst']  for model in models],
	'train time':          [model['train_time']  for model in models],
	'Inference time':      [model['inference_time']  for model in models],
}).set_index('Algorithm').sort_values(by='Test Score', ascending=False)

print("Traditional ML Models Performance:")
print(ml_results_df)

## Classification Report

In [ ]:
for model in models:
	print(model['name'])
	print(model['cr'])

## Hyperparemeter tuning for best performing traditional ml models: SVC, logistic regresiion and Extratress classifier

###Grid search

In [ ]:
param_grid_svc = {
    'C': [0.01, 0.1, 1, 10],  # Regularization strength
    'kernel': ['linear', 'rbf', 'poly'],  # Kernel type
    'gamma': ['scale', 'auto']  # Kernel coefficient for ‘rbf’, ‘poly’
}

grid_svc = GridSearchCV(SVC(), param_grid_svc, cv=5, scoring='accuracy', n_jobs=-1)
grid_svc.fit(X_trn, y_trn)

print("Best parameters for SVC:", grid_svc.best_params_)
print("Best SVC Score:", grid_svc.best_score_)

In [ ]:
param_grid_logreg = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'penalty': ['l1', 'l2'],  # Type of regularization
    'solver': ['liblinear', 'saga']  # Solver type (for L1, use 'liblinear' or 'saga')
}

grid_logreg = GridSearchCV(LogisticRegression(max_iter=5000), param_grid_logreg, cv=5, scoring='accuracy', n_jobs=-1)
grid_logreg.fit(X_trn, y_trn)

print("Best parameters for Logistic Regression:", grid_logreg.best_params_)
print("Best Logistic Regression Score:", grid_logreg.best_score_)

###Randomised search

In [ ]:
param_dist_extratrees = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'max_depth': [None, 10, 20],  # Maximum depth of trees
    'min_samples_split': [2, 5, 10],  # Minimum samples to split
    'min_samples_leaf': [1, 2, 4],  # Minimum samples per leaf
    'max_features': ['sqrt', 'log2', None]  # Features to consider per split
}
# Use RandomizedSearchCV
random_extratrees = RandomizedSearchCV(
    ExtraTreesClassifier(),
    param_distributions=param_dist_extratrees,
    n_iter=20,  # Number of random parameter combinations to try
    cv=3,  # Reduce cross-validation folds to speed up training
    scoring='accuracy',
    n_jobs=-1,  # Use all available CPU cores
    random_state=42  # For reproducibility
)
# Fit the model
random_extratrees.fit(X_trn, y_trn)
# Print best parameters and score
print("Best parameters for Extra Trees:", random_extratrees.best_params_)
print("Best Extra Trees Score:", random_extratrees.best_score_)

#BERT

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
#Spliting data FIRST to prevent leakage  which happned in the last code
# First split: 80% train, 20% test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    sampled_df['reviewText'],
    sampled_df['scoreSentiment'],
    test_size=0.2,
    stratify=sampled_df['scoreSentiment'],
    random_state=99
)

In [ ]:
# Second split: 80% of original -> 80% train (64% total), 20% validation (16% total)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts,
    train_labels,
    test_size=0.2,
    stratify=train_labels,
    random_state=99
)

In [ ]:
#Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 128

# 3. Tokenization function
def encode_texts(texts):
    return tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='tf'
    )
# Encode all splits(imp for bert)
train_encodings = encode_texts(train_texts)
val_encodings = encode_texts(val_texts)
test_encodings = encode_texts(test_texts)

In [ ]:
#Convert to TensorFlow Dataset for better performance
def create_dataset(encodings, labels):
    return tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask']
        },
        labels
    )).batch(32).prefetch(tf.data.AUTOTUNE)

train_dataset = create_dataset(train_encodings, train_labels)
val_dataset = create_dataset(val_encodings, val_labels)
test_dataset = create_dataset(test_encodings, test_labels)

In [ ]:
#Model setup
model = TFBertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)

In [ ]:
#Configure training
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# 7. Train the model
# Function to measure training time
def time_bert_training(model, train_dataset, val_dataset, epochs=5):
    # Start timing for training
    train_start = time.time()

    # Train the model
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=epochs,
        verbose=1
    )

    train_end = time.time()
    train_time = train_end - train_start

    print(f'BERT Training Time: {train_time:.2f} seconds')
    return history, train_time

# Function to measure inference time
def time_bert_inference(model, test_dataset):
    # Start timing for inference
    inference_start = time.time()

    # Run prediction
    predictions = model.predict(test_dataset)

    inference_end = time.time()
    inference_time = inference_end - inference_start

    # Get the number of test examples
    num_examples = sum(batch[1].shape[0] for batch in test_dataset)
    per_example_time = inference_time / num_examples

    print(f'BERT Inference Time: {inference_time:.2f} seconds')
    print(f'Time per example: {per_example_time*1000:.2f} ms')

    return predictions, inference_time

In [ ]:
#Full evaluation
history, bert_train_time = time_bert_training(model, train_dataset, val_dataset)
predictions, bert_inference_time = time_bert_inference(model, test_dataset)
y_pred = predictions.logits.argmax(axis=1)

print("\nClassification Report:")
print(classification_report(test_labels, y_pred))

#Confusion matrix
from sklearn.metrics import confusion_matrix
print("\nConfusion Matrix:")
print(confusion_matrix(test_labels, y_pred))

In [ ]:
# After the training code block:

# Get scores for FINAL MODEL
train_loss, train_acc = model.evaluate(train_dataset, verbose=0)
val_loss, val_acc = model.evaluate(val_dataset, verbose=0)
test_loss, test_acc = model.evaluate(test_dataset, verbose=0)

# Create comprehensive results table
results_df = pd.DataFrame({
    'Algorithm': ['BERT'],
    'Train Score': [train_acc],
    'Test Score': [test_acc],
    'train time': [bert_train_time],
    'Inference time': [bert_inference_time]
})
print("\nModel Performance:")
print(results_df.round(4))

# Optional: Compare with last epoch's training accuracy
final_epoch_train_acc = history.history['accuracy'][-1]
print(f"\nLast Epoch Training Accuracy: {final_epoch_train_acc:.4f}")

# Hyper tuning for bert


In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define hyperparameter grid
learning_rates = [5e-5, 4e-5, 3e-5, 2e-5]
batch_sizes = [16, 32]

# Ensure dataset columns exist
assert 'reviewText' in sampled_df and 'scoreSentiment' in sampled_df, "Dataset columns missing!"

# Split data into train / val / test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    sampled_df['reviewText'].tolist(),
    sampled_df['scoreSentiment'].tolist(),
    test_size=0.2,
    stratify=sampled_df['scoreSentiment'],
    random_state=99
)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts,
    train_labels,
    test_size=0.2,
    stratify=train_labels,
    random_state=99
)

# Tokenization helper
def encode_texts(texts, tokenizer, max_length=128):
    return tokenizer(
        texts, padding=True, truncation=True, max_length=max_length, return_tensors="tf"
    )

# Encode all splits
train_encodings = encode_texts(train_texts, tokenizer)
val_encodings   = encode_texts(val_texts, tokenizer)
test_encodings  = encode_texts(test_texts, tokenizer)

# Create tf.data.Dataset
def create_dataset(encodings, labels, batch_size):
    ds = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask']
        },
        labels
    ))
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Grid search over all combinations
best_val_accuracy = 0.0
best_params = {}
trial_num = 0

total_trials = len(learning_rates) * len(batch_sizes)
print(f"Starting grid search over {total_trials} trials...\n")

for lr in learning_rates:
    for batch_size in batch_sizes:
        trial_num += 1
        print(f"Trial {trial_num}/{total_trials}: lr={lr}, batch_size={batch_size}")

        # Prepare datasets
        train_ds = create_dataset(train_encodings, train_labels, batch_size)
        val_ds   = create_dataset(val_encodings,   val_labels,   batch_size)
        test_ds  = create_dataset(test_encodings,  test_labels,  batch_size)

        # Initialize and compile model
        model = TFBertForSequenceClassification.from_pretrained(
            "bert-base-uncased", num_labels=2
        )
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(
            optimizer=optimizer,
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )

        # Train
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=5,
            verbose=1
        )

        # Evaluate
        max_train_acc = max(history.history['accuracy'])
        max_val_acc   = max(history.history['val_accuracy'])
        test_loss, test_acc = model.evaluate(test_ds, verbose=0)

        print(f"  --> Train Acc: {max_train_acc:.4f}, Val Acc: {max_val_acc:.4f}, Test Acc: {test_acc:.4f}\n")

        # Update best
        if max_val_acc > best_val_accuracy:
            best_val_accuracy = max_val_acc
            best_params = {
                'learning_rate': lr,
                'batch_size': batch_size,
                'best_train_accuracy': max_train_acc,
                'best_val_accuracy': max_val_acc,
                'test_accuracy': test_acc
            }

# Final results
print("Grid search complete. Best hyperparameters and performance:", best_params)

#Comparing traditional methods with BERT

In [ ]:
# Create one consistent combined dataframe with ALL metrics
ml_df = ml_results_df.reset_index().rename(columns={'Algorithm': 'Model'})
bert_df = results_df.rename(columns={'Algorithm': 'Model'})
#Concatenate with all columns preserved
combined_results = pd.concat([ml_df, bert_df], ignore_index=True)
print(combined_results)

In [ ]:
# Melt the dataframe to long format for grouped bars
melted_results = pd.melt(
    combined_results,
    id_vars='Model',
    value_vars=['Train Score', 'Test Score'],
    var_name='Score Type',
    value_name='Accuracy'
)

# Sort models by test accuracy for cleaner order)
model_order = combined_results.sort_values(by='Test Score', ascending=False)['Model']
melted_results['Model'] = pd.Categorical(melted_results['Model'], categories=model_order, ordered=True)

#Plotting
plt.figure(figsize=(14, 7))
sns.set(style='whitegrid')

bar = sns.barplot(
    data=melted_results,
    x='Model',
    y='Accuracy',
    hue='Score Type',
    palette='viridis'
)

# Annotate bars
for p in bar.patches:
    height = p.get_height()
    bar.annotate(
        f'{height:.2f}',
        (p.get_x() + p.get_width() / 2, height + 0.01),
        ha='center',
        fontsize=9,
        weight='bold'
    )

# Customize the plot
plt.title('Train vs Test Accuracy Comparison of ML Models vs BERT', fontsize=16, weight='bold')
plt.ylabel('Accuracy', fontsize=12)
plt.xlabel('Model', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.ylim(0, 1.05)
plt.legend(title='Score Type', loc='upper right')
plt.tight_layout()
plt.show()

##Comparing Genearalisation across all the models

In [ ]:
#Compute the gap and sort
combined_results['Gen. Gap'] = combined_results['Train Score'] - combined_results['Test Score']
combined_results = combined_results.set_index('Model').sort_values('Gen. Gap', ascending=False)
print(combined_results)

In [ ]:
plt.figure(figsize=(12, 8))
# Plot generalization gap
sns.barplot(data=combined_results.reset_index(), x='Model', y='Gen. Gap', palette='coolwarm')
plt.title('Generalization Gap by Model (Train - Test Accuracy)', fontsize=14)
plt.xlabel('Model', fontsize=12)
plt.ylabel('Gap', fontsize=12)
plt.xticks(rotation=45, ha='right')
for i, v in enumerate(combined_results['Gen. Gap']):
    plt.text(
        i,
        v + 0.005,
        f'{v:.3f}',
        ha='center',
        fontsize=9
    )
plt.tight_layout()
plt.show()

##Comparing compuational time across all the models

In [ ]:
# Sort by training time
train_time_order = combined_results.sort_values('train time').index.tolist()
train_time_df = combined_results.copy()
train_time_df['Model'] = pd.Categorical(train_time_df.index, categories=train_time_order, ordered=True)

In [ ]:
# Sort by inference time
inference_time_order = combined_results.sort_values('Inference time').index.tolist()
inference_time_df = combined_results.copy()
inference_time_df['Model'] = pd.Categorical(inference_time_df.index, categories=inference_time_order, ordered=True)

In [ ]:
plt.figure(figsize=(16, 8))
# subplots
fig, (ax2, ax3) = plt.subplots(1, 2, figsize=(16, 6))
# Training time plot (log scale)
sns.barplot(data=train_time_df, x='Model', y='train time', ax=ax2, palette='viridis')
ax2.set_title('Training Time Comparison (Log Scale)', fontsize=14)
ax2.set_xlabel('Model', fontsize=12)
ax2.set_ylabel('Time (seconds)', fontsize=12)
ax2.set_yscale('log')
ax2.tick_params(axis='x', rotation=45)
for p in ax2.patches:
    height = p.get_height()
    ax2.annotate(
        f'{height:.2f}s',
        (p.get_x() + p.get_width() / 2., height*1.05),
        ha='center',
        va='bottom',
        fontsize=8
    )

# Inference time plot (log scale)
sns.barplot(data=inference_time_df, x='Model', y='Inference time', ax=ax3, palette='plasma')
ax3.set_title('Inference Time Comparison (Log Scale)', fontsize=14)
ax3.set_xlabel('Model', fontsize=12)
ax3.set_ylabel('Time (seconds)', fontsize=12)
ax3.set_yscale('log')
ax3.tick_params(axis='x', rotation=45)
for p in ax3.patches:
    height = p.get_height()
    ax3.annotate(
        f'{height:.3f}s',
        (p.get_x() + p.get_width() / 2., height*1.05),
        ha='center',
        va='bottom',
        fontsize=8
    )

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.set_style("whitegrid")

combined_results = combined_results.reset_index()

# Create efficiency scatter plot
plt.scatter(
    combined_results['Inference time'],
    combined_results['Test Score'],
    s=100,
    alpha=0.7,
    c=np.arange(len(combined_results)),
    cmap='viridis'
)

# Add algorithm labels to the points
for i, alg in enumerate(combined_results['Model']):
    plt.annotate(
        alg,
        (combined_results['Inference time'].iloc[i]*1.1, combined_results['Test Score'].iloc[i]),
        fontsize=9
    )

plt.xscale('log')
plt.xlabel('Inference Time (seconds) - Log Scale', fontsize=12)
plt.ylabel('Test Accuracy', fontsize=12)
plt.title('Efficiency Comparison: Accuracy vs Inference Time', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

#Loading BERT

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# === Hyperparameters (Best-found) ===
BEST_PARAMS = {
    'learning_rate': 3e-5,
    'batch_size': 32,
    'epochs': 5,
    'max_length': 128,
    'model_name': 'bert-base-uncased',
    'save_dir': './best_bert_model'
}


df = sampled_df.copy()
df['label'] = df['scoreSentiment'].astype(int)

# First split: train+val and test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['reviewText'], df['label'], test_size=0.2, stratify=df['label'], random_state=42
)
# Second split: train and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, stratify=train_labels, random_state=42
)

# === Tokenizer and Encoding ===
tokenizer = BertTokenizer.from_pretrained(BEST_PARAMS['model_name'])

def encode_texts(texts, max_length):
    return tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='tf'
    )

train_enc = encode_texts(train_texts, BEST_PARAMS['max_length'])
val_enc   = encode_texts(val_texts, BEST_PARAMS['max_length'])
test_enc  = encode_texts(test_texts, BEST_PARAMS['max_length'])

# === Create TensorFlow Datasets ===
def make_dataset(encodings, labels, batch_size):
    ds = tf.data.Dataset.from_tensor_slices((
        {'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask']},
        labels.values
    ))
    return ds.batch(32).prefetch(tf.data.AUTOTUNE)

train_ds = make_dataset(train_enc, train_labels, BEST_PARAMS['batch_size'])
val_ds   = make_dataset(val_enc,   val_labels,   BEST_PARAMS['batch_size'])
test_ds  = make_dataset(test_enc,  test_labels,  BEST_PARAMS['batch_size'])

# === Build and Compile Model ===
model = TFBertForSequenceClassification.from_pretrained(
    BEST_PARAMS['model_name'],
    num_labels=2
)
optimizer = tf.keras.optimizers.Adam(learning_rate=BEST_PARAMS['learning_rate'])
loss_fn   = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics   = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]

model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

# === Train ===
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=BEST_PARAMS['epochs']
)

# === Evaluate on Test Set ===
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

# === Save Model and Tokenizer ===
os.makedirs(BEST_PARAMS['save_dir'], exist_ok=True)
model.save_pretrained(BEST_PARAMS['save_dir'])
tokenizer.save_pretrained(BEST_PARAMS['save_dir'])
print(f"Model and tokenizer saved to: {BEST_PARAMS['save_dir']}")


In [ ]:
# 1. Compress the folder into a single .zip
!zip -r best_bert_model.zip best_bert_model

# 2. Bring up a browser download dialog
from google.colab import files
files.download('best_bert_model.zip')